Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All).

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE", as well as your name and collaborators below:

---

# Homework 1 - Python
### Luca de Alfaro, CSE 183

In this homework, you have to fill in all the places that say `### YOUR CODE HERE`.  Above each code section where you have to insert your code, I wrote how many lines of code used my solution.  This is just an indication; your solution can be shorter or longer, and it is fine provided it works. 
I note how many lines it took me to solve the problem just to give you some indication of the complexity of the problem. 

There are three groups questions in this homework: 

* Subclassing
* Fibonacci
* Decorators

Once you are done, you need to download the notebook **in .ipynb format** and upload it [to this form](https://docs.google.com/forms/d/e/1FAIpQLSfB77Zt03Isd7FFt2No_hfN9z34pdusceBChJbHDuizCkOhug/viewform?usp=sf_link).

## Question: Subclassing. Implementing a bounded cache.

Here is a class implementing a cache.  The cache has two methods:

*   `set`: sets an element.  
*   `get`: gets an element. 

As it is, this cache is just a wrapper over a dictinary, but we will ask you to subclass it to give it more functionality in the following.



In [0]:
class Cache(object):

    def __init__(self):
        self.d = {}

    def set(self, k, v):
        """Sets the key k to value v in the cache."""
        self.d[k] = v

    def get(self, k):
        """Gets the value of key k from the cache, or None if the
        key is not present."""
        return self.d.get(k)

    def remove(self, k):
        """Removes key k from the cache."""
        if k in self.d:
            del self.d[k]

    def len(self):
        return len(self.d)


As a partial step, extend the Cache class to an OrderedCache one, where you can ask for the keys in the order in which they have been accessed, where an access is either a get or a set.  Simply add to the class a key list that stores this order, for instance, from oldest to newest key.

In [0]:
### OrderedCache

class OrderedCache(Cache):

    def __init__(self):
        super().__init__()
        # 1 line
        # YOUR CODE HERE
        self.orderL=[]

    def _accessed(self, k):
        """This function notes that the key k has been accessed.
        NOTE: A key counts as accessed only if it actually is in
        the dictionary, not if someone tries to get its value, but
        it does not belong to the dictionary!
        """
        # 4 lines
        # YOUR CODE HERE
        if k in self.d:
          if k in self.orderL:
            self.orderL.remove(k)
          self.orderL.append(k)

    def set(self, k, v):
        super().set(k, v)
        self._accessed(k)

    def get(self, k):
        self._accessed(k)
        return super().get(k)

    def remove(self, k):
        """We remove the key from the list of keys before calling
        super().remove"""
        # Remove k from the key list, if present.
        # 2 lines
        # YOUR CODE HERE
        if k in self.d:
          self.orderL.remove(k)
        super().remove(k)

    def len(self, k):
        return len(self.d)

    @property
    def oldest_key(self):
        """Returns the oldest accessed key, or None if no key has ever been
        accessed."""
        # 1 line
        # YOUR CODE HERE
        return self.orderL[0] or None

    @property
    def newest_key(self):
        """Returns the most newly accessed key, or None if no key has ever
        been accessed."""
        # 1 line
        # YOUR CODE HERE
        return self.orderL[-1] or None


In [0]:
### Here are some tests (10 points)

c = OrderedCache()

c.set('a', 2)
c.set('b', 3)
assert c.oldest_key == 'a'
assert c.newest_key == 'b'

# If we access 'c', which does not exist, the newest key is still b.
assert c.get('c') == None
assert c.oldest_key == 'a'
assert c.newest_key == 'b'

# This works also for keys that are not strings...
c.set(5, 6)
assert c.newest_key == 5



In [0]:
### Some more tests (10 points)

# If we now access 'a', then it becomes the newest key.
assert c.get('a') == 2
assert c.oldest_key == 'b'
assert c.newest_key == 'a'



Now we want to extend OrderedCache to BoundedCache.  A BoundedCache is a cache that contains at most a given number of elements.  If you try to add one more key to it, the oldest key is removed from the cache. 

In [0]:
### BoundedCache

class BoundedCache(OrderedCache):

    def __init__(self, size_limit=4):
        """Creates a bounded cache with a given size limit.
        If more than size_limit elements are added, the oldest-accessed
        elements are deleted from the cache."""
        super().__init__()
        self.size_limit = size_limit

    def set(self, k, v):
        """Sets key k to map to element v.
        If k is already in the cache, the mapping should be simply updated.
        If k is not in the cache, and the cache already
        has reached maximum size, deletes the oldest accessed mapping
        before adding the k:v one."""
        # 3 lines
        # YOUR CODE HERE
        if k in self.d:
          super().set(k,v)
        else:
          if super().len(k) < self.size_limit:
            super().set(k,v)
          else:
            super().remove(super().oldest_key)
            super().set(k,v)

In [0]:
### Some tests for bounded cache (10 points)

c = BoundedCache(2)
c.set('a', 1)
c.set('b', 2)
c.set('c', 3)
assert c.get('a') == None
assert c.get('b') == 2

c = BoundedCache(3)
c.set('a', 1)
c.set('b', 2)
c.set('c', 3)
c.get('a')
c.set('d', 4)
assert c.get('a') == 1
assert c.get('b') == None

c = BoundedCache(3)
c.set(1, 1)
c.set(2, 2)
c.set(3, 3)
c.get(1)
c.set(4, 4)
c.set(5, 5)
assert c.get(4) == 4
assert c.get(3) == None
assert c.get(1) == 1
assert c.get(2) == None



## Question: A Better Fibonacci Function

The Fibonacci function we wrote is very inefficient:

In [0]:
def fibo(n):
    return n if n < 2 else fibo(n - 1) + fibo(n - 2)


Can you write a more efficient version of it?

In [0]:
### Better Fibonacci
### this method uses a list to remember the previous answers and add them using the index -1 and -2 for last and second last thing in the list
def efibo(n):
    fibList=[0,1]
    for x in range(2,n+1):
      fibList.append(fibList[-1]+fibList[-2])
    return fibList[n]

In [26]:
efibo(20), fibo(20)


(6765, 6765)

In [0]:
### Correctness of efibo (10 point)

for n in range(20):
    assert fibo(n) == efibo(n)



In [0]:
### Efficiency of efibo (10 points)

assert efibo(40) == 102334155



## Question: Decorators

Write a decorator `positivize1` that, when decorating a function `f` of one argument, makes sure that the argument fed to `f` is nonnegative by taking its absolute value.   For instance, if we do:

    @positivize1
    def f(x):
        return x + 1

then `f(-1)` should return `2`, because we take the absolute value of `-1` before passing it to `f`.

In [0]:
### positivize1

def positivize1(func):
    # 3 lines
    # YOUR CODE HERE
    def absolute(*args,**kwargs):
      args=[abs(x) for x in args]
      return func(*args)
    return absolute

In [0]:
### Tests for `positivize1` (10 points)

@positivize1
def f(x):
    return x + 1
assert f(1) == f(-1)
assert f(-2) == 3



Now write a `@positivize` decorator, that works for functions of any number of arguments, including keyword arguments.  We assume that the value of all the arguments must be numbers, of course. 

In [0]:
### positivize

def positivize(func):
    # 5 lines (can be squeezed to 3 lines)
    # YOUR CODE HERE
    def absolute(*args,**kwargs):
      args=[abs(x) for x in args]
      for key, value in kwargs.items():
        kwargs[key]=abs(value)
      return func(*args,**kwargs)
    return absolute

In [0]:
### Tests for `positivize` (10 points)

@positivize
def add(x, y, inc=0):
    return x + y + inc

assert add(1, 2, inc=1) == 4
assert add(2, 2, inc=-1) == 5

@positivize
def quack(x, y, z, inc=0, diff=1):
    return x + y * z + (inc - diff) * 2

assert quack(1, 2, 3) == 5



Finally, write a decorator with arguments so that if you apply:

    @raiseargs(3)

above a function that takes numeric positional or keyword arguments, all the arguments are raised to the specified power (in this case, 3) before being passed to the decorated function.

In [0]:
### raiseargs

def raiseargs(n):
    """Decorator that raises to the n-th power all arguments
    (positional, or keyword arguments) of a given function."""
    # 7 lines
    # YOUR CODE HERE
    def inner1(func):
      def func_wrapper(*args,**kwargs):
        args=[x**n for x in args]
        for key, value in kwargs.items():
          kwargs[key]=value**n
        return func(*args,**kwargs)
      return func_wrapper
    return inner1


In [0]:
### Tests for raiseargs (10 points)

@raiseargs(2)
def f(x, y):
    return x + y

assert f(2, 3) == 13 # 4 + 9

@raiseargs(3)
def g(x, y=1):
    return x + y + 1

assert g(2, y=2) == 17 # 8 + 8 + 1

